In [17]:
import requests
import pandas as pd
import openpyxl

# Read in list of assets of interest and convert to list
assetFrame = pd.read_excel('assetList.xlsx')
assetList = assetFrame['Asset'].tolist()

# Initialize an empty list to store the data
data_list = []

for asset in assetList:

    # Initial URL for the first API call
    base_url = "https://clinicaltrials.gov/api/v2/studies"
    params = {
        "query.term": asset,
        "pageSize": 100
    }
    # Assign asset name from parameter
    assetName = params['query.term']

    # Loop until there is no nextPageToken
    while True:
        # Print the current URL (for debugging purposes); Access by looping parameters
        print("Fetching data from:", base_url + '?' + '&'.join([f"{k}={v}" for k, v in params.items()]))
        
        # Send a GET request to the API
        response = requests.get(base_url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()  # Parse JSON response
            studies = data.get('studies', [])  # Extract the list of studies, pass empty parameters

            # Loop through each study and extract specific information
            for study in studies:
                # Safely access nested keys, Indicates Unknown if unavailable 
                nctId = study['protocolSection']['identificationModule'].get('nctId', 'Unknown')
                studyTitle = study['protocolSection']['identificationModule'].get('officialTitle', 'Unknown')
                sponsor = study['protocolSection']['sponsorCollaboratorsModule'].get('leadSponsor', {}).get('name', 'Unknonw Lead Sponsor')
                overallStatus = study['protocolSection']['statusModule'].get('overallStatus', 'Unknown')
                enrollment = study['protocolSection']['designModule'].get('enrollmentInfo',{}).get('count', 'Unknown')
                startDate = study['protocolSection']['statusModule'].get('startDateStruct', {}).get('date', 'Unknown Date')
                conditions = ', '.join(study['protocolSection']['conditionsModule'].get('conditions', ['No conditions listed']))
                acronym = study['protocolSection']['identificationModule'].get('acronym', 'Unknown')

                # Extract interventions safely
                interventions_list = study['protocolSection'].get('armsInterventionsModule', {}).get('interventions', [])
                interventions = ', '.join([intervention.get('name', 'No intervention name listed') for intervention in interventions_list]) if interventions_list else "No interventions listed"

                # Extract outcomes safely
                primaryOutcomes_list = study['protocolSection'].get('outcomesModule', {}).get('primaryOutcomes', [])
                primaryOutcomes = ', '.join([measure.get('measure', 'No primary outcome indicated') for measure in primaryOutcomes_list]) if primaryOutcomes_list else "No primary outcomes listed"
                secondaryOutcomes_list = study['protocolSection'].get('outcomesModule', {}).get('secondaryOutcomes', [])
                secondaryOutcomes = ', '.join([measure.get('measure', 'No secomdary outcomes indicated') for measure in secondaryOutcomes_list]) if secondaryOutcomes_list else "No secondary outcomes listed"
                
                # Extract locations safely
                locations_list = study['protocolSection'].get('contactsLocationsModule', {}).get('locations', [])
                locations = ', '.join([f"{location.get('city', 'No City')} - {location.get('country', 'No Country')}" for location in locations_list]) if locations_list else "No locations listed"
                
                # Extract dates and phases
                primaryCompletionDate = study['protocolSection']['statusModule'].get('primaryCompletionDateStruct', {}).get('date', 'Unknown Date')
                studyFirstPostDate = study['protocolSection']['statusModule'].get('studyFirstPostDateStruct', {}).get('date', 'Unknown Date')
                lastUpdatePostDate = study['protocolSection']['statusModule'].get('lastUpdatePostDateStruct', {}).get('date', 'Unknown Date')
                studyType = study['protocolSection']['designModule'].get('studyType', 'Unknown')
                phases = ', '.join(study['protocolSection']['designModule'].get('phases', ['Not Available']))

                # Append the data to the list as a dictionary
                data_list.append({
                    "NCT ID": nctId,
                    "Asset": assetName,
                    "Title": studyTitle,
                    "Lead Sponsor": sponsor,
                    "Acronym": acronym,
                    "Overall Status": overallStatus,
                    "Enrollment": enrollment,
                    "Start Date": startDate,
                    "Conditions": conditions,
                    "Interventions": interventions,
                    "Primary Outcome(s)": primaryOutcomes,
                    "Secondary Outcome(s)": secondaryOutcomes,
                    "Locations": locations,
                    "Primary Completion Date": primaryCompletionDate,
                    "Study First Post Date": studyFirstPostDate,
                    "Last Update Post Date": lastUpdatePostDate,
                    "Study Type": studyType,
                    "Phases": phases
                })

            # Check for nextPageToken and update the params or break the loop
            nextPageToken = data.get('nextPageToken')
            if nextPageToken:
                params['pageToken'] = nextPageToken  # Set the pageToken for the next request
            else:
                break  # Exit the loop if no nextPageToken is present
        else:
            print("Failed to fetch data. Status code:", response.status_code)
            break

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Optionally, save the DataFrame to a CSV file
df.to_csv("oldClinicalTrialsData.csv", index=False)

# Print the DataFrame
df

Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=Bitopertin&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=GSBR-1290&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=RP-1&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=Ritlecitinib&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=MK-0616&pageSize=100


,NCT ID,Asset,Title,Lead Sponsor,Acronym,Overall Status,Enrollment,Start Date,Conditions,Interventions,Primary Outcome(s),Secondary Outcome(s),Locations,Primary Completion Date,Study First Post Date,Last Update Post Date,Study Type,Phases
0,NCT01636492,Bitopertin,"Single Oral Ascending Dose Study of Safety, To...",Hoffmann-La Roche,Unknown,COMPLETED,71,2005-11,Healthy Volunteer,"bitopertin, placebo",Safety: Incidence of adverse events,Pharmacokinetics: Area under the concentration...,Christchurch - New Zealand,2006-08,2012-07-10,2016-11-02,INTERVENTIONAL,PHASE1
1,NCT05308472,Bitopertin,"(AURORA) A Randomized, Double-blind, Placebo-C...","Disc Medicine, Inc",Unknown,ACTIVE_NOT_RECRUITING,75,2022-10-31,Erythropoietic Protoporphyria,"DISC-1459, DISC-1459, Placebo, DISC-1459",Percent change from baseline in whole blood me...,Total hours of sunlight exposure to skin on da...,"Birmingham - United States, San Francisco - Un...",2024-02-20,2022-04-04,2024-04-15,INTERVENTIONAL,PHASE2
2,NCT01192880,Bitopertin,"A Phase III, Multi-Center, Randomized, 24 Week...",Hoffmann-La Roche,Unknown,COMPLETED,625,2010-11,Schizophrenia,"Placebo, Bitopertin, Antipsychotics",Mean Change From Baseline in the Positive and ...,Mean Change from Baseline in the Personal and ...,"Costa Mesa - United States, Escondido - United...",2014-07,2010-09-01,2017-02-15,INTERVENTIONAL,PHASE3
3,NCT05883748,Bitopertin,"An Open-Label, Long-Term Study to Investigate ...","Disc Medicine, Inc",Unknown,ENROLLING_BY_INVITATION,200,2023-08-31,Erythropoietic Protoporphyria,"DISC-1459, DISC-1459",Incidence of treatment-emergent adverse events...,Change from baseline in daily daylight toleran...,"Birmingham - United States, San Francisco - Un...",2028-06,2023-06-01,2024-02-22,INTERVENTIONAL,"PHASE2, PHASE3"
4,NCT02019290,Bitopertin,"A SINGLE CENTER, OPEN LABEL, FIXED SEQUENCE ST...",Hoffmann-La Roche,Unknown,WITHDRAWN,0,2014-02-10,Healthy Volunteer,"Midazolam, bitopertin",Change in plasma area under the concentration-...,Change in 1'-hydroxymidazolam plasma AUC after...,No locations listed,2014-03-31,2013-12-24,2017-10-25,INTERVENTIONAL,PHASE1
5,NCT05828108,Bitopertin,"A Phase I/II, Intra-Patient Dose-Escalation St...","National Heart, Lung, and Blood Institute (NHLBI)",Unknown,RECRUITING,30,2023-07-25,Steroid-refractory Diamond-Blackfan Anemia (DBA),Bitopertin,Rate of response,"Rate of relapse, Long-term safety of drug, Tol...",Bethesda - United States,2029-05-01,2023-04-25,2024-04-10,INTERVENTIONAL,"PHASE1, PHASE2"
6,NCT03271541,Bitopertin,"A Phase II, Single Arm, Multicenter, Proof-of-...",Hoffmann-La Roche,Unknown,COMPLETED,12,2017-10-26,Beta-Thalassemia,Bitopertin,Safety Outcome: Percentage of Participants wit...,"Apparent Clearance of Bitopertin, Volume of Di...","Genova - Italy, Milano - Italy, Baabda - Leban...",2018-06-29,2017-09-05,2018-10-05,INTERVENTIONAL,PHASE2
7,NCT01192906,Bitopertin,"A Phase III, Multi-center, Randomized, 24 Week...",Hoffmann-La Roche,Unknown,COMPLETED,624,2010-12,Schizophrenia,"Placebo, bitopertin [RO4917838], bitopertin [R...",Efficacy (PANSS negative symptoms factor score...,No secondary outcomes listed,"Carson - United States, Oceanside - United Sta...",2014-07,2010-09-01,2016-11-02,INTERVENTIONAL,PHASE3
8,NCT01234779,Bitopertin,"A Phase II, Multi-center, Randomized, 4-week, ...",Hoffmann-La Roche,Unknown,COMPLETED,301,2011-02,Schizophrenia,"bitopertin [RO4917838], bitopertin [RO4917838]...",Change in Positive and Negative Syndrome Scale...,"Clinical response, defined as at least 30% or ...","Little Rock - United States, Cerritos - United...",2012-09,2010-11-04,2016-11-02,INTERVENTIONAL,PHASE2
9,NCT01235559,Bitopertin,"Phase III, Multi-center, Randomized, 12-week, ...",Hoffmann-La Roche,Unknown,COMPLETED,604,2010-12,Schizophrenia,"Placebo, bitopertin [RO4917838] level 1, bitop...",Positive symptoms factor score assessed by Pos...,Symptom domains of schizophrenia using Positiv...,"Costa Mesa - United States, Escondido - United...",2015-03,2010-11